In [28]:
import pandas as pd
import numpy as np
from datetime import datetime
import numpy as np
import os
import csv
import pandas as pd
import Gaussian
import Parameters
import Util
import matplotlib.pyplot as plt
from Dependency import Dependency

In [9]:
d = Dependency()
path = os.getcwd()[:-5] + 'Model Dependencies/'
read_path = path + 'Region_Mobility_Report_CSVs/'
path_2020 = read_path + '2020_CA_Region_Mobility_Report.csv'
path_2021 = read_path + '2021_CA_Region_Mobility_Report.csv'
path_2022 = read_path + '2022_CA_Region_Mobility_Report.csv'
df1 = pd.read_csv(path_2020)
df2 = pd.read_csv(path_2021)
df3 = pd.read_csv(path_2022)

In [10]:
df = pd.concat([df1, df2, df3])

In [11]:
print(df)

      country_region_code country_region sub_region_1 sub_region_2  \
0                      CA         Canada          NaN          NaN   
1                      CA         Canada          NaN          NaN   
2                      CA         Canada          NaN          NaN   
3                      CA         Canada          NaN          NaN   
4                      CA         Canada          NaN          NaN   
...                   ...            ...          ...          ...   
74211                  CA         Canada        Yukon          NaN   
74212                  CA         Canada        Yukon          NaN   
74213                  CA         Canada        Yukon          NaN   
74214                  CA         Canada        Yukon          NaN   
74215                  CA         Canada        Yukon          NaN   

       metro_area iso_3166_2_code  census_fips_code  \
0             NaN             NaN               NaN   
1             NaN             NaN               N

In [24]:
groups = df.groupby('iso_3166_2_code')
Ontario = groups.get_group('CA-ON')
Ontario['date'] = pd.to_datetime(Ontario['date'])
max_date = (Ontario['date'].max() - Parameters.OUTBREAK_FIRST_DAY).days
min_date = (Ontario['date'].min() - Parameters.OUTBREAK_FIRST_DAY).days
print(max_date, min_date)

1004 31


C:\Users\loveb\AppData\Local\Temp\ipykernel_37928\1164355118.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ontario['date'] = pd.to_datetime(Ontario['date'])


In [33]:
mobility = np.ones(shape=(6, d.total_days))

In [38]:
retail = Ontario['retail_and_recreation_percent_change_from_baseline'].to_numpy()
grocery = Ontario['grocery_and_pharmacy_percent_change_from_baseline'].to_numpy()
park = Ontario['parks_percent_change_from_baseline'].to_numpy()
trainsit = Ontario['transit_stations_percent_change_from_baseline'].to_numpy()
workplace = Ontario['workplaces_percent_change_from_baseline'].to_numpy()
residential = Ontario['residential_percent_change_from_baseline'].to_numpy()
mobility[0][min_date:max_date+1] = retail
mobility[1][min_date:max_date+1] = grocery
mobility[2][min_date:max_date+1] = park
mobility[3][min_date:max_date+1] = trainsit
mobility[4][min_date:max_date+1] = workplace
mobility[5][min_date:max_date+1] = residential
mobility = mobility.T